<a href="https://colab.research.google.com/github/jordantab/11785-assignment-part2s/blob/main/HW1P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 28 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [2]:
!pip install torchsummaryX==1.1.0 wandb --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 113.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00


In [3]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [4]:
''' If you are using colab, you can import google drive to save model checkpoints in a folder
    If you want to use it, uncomment the two lines below
'''
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [12]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"jordantab","key":"6012317c8cbe2756d1bf461f146b1d6e"}')
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

  Using cached kaggle-1.5.8-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [13]:
# commands to download data from kaggle
!kaggle competitions download -c 11785-hw1p2-f24

!unzip -qo /content/11785-hw1p2-f24.zip -d '/content'

100% 3.98G/3.98G [03:36<00:00, 20.5MB/s]
100% 3.98G/3.98G [03:36<00:00, 19.8MB/s]


# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [14]:
# Dataset class to load train and validation data

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "train-clean-100"): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes

        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir       = os.path.join(root, partition, "mfcc")
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        self.transcript_dir = os.path.join(root, partition, "transcript")

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names          = sorted(os.listdir(self.mfcc_dir))
        # TODO: List files in self.transcript_dir using os.listdir in sorted order
        transcript_names    = sorted(os.listdir(self.transcript_dir))

        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names)

        self.mfccs, self.transcripts = [], []

        # TODO: Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc        = np.load(os.path.join(self.mfcc_dir, mfcc_names[i]))
            # print(mfcc.shape)
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc = (mfcc - np.mean(mfcc)) / np.std(mfcc)
        #   Load the corresponding transcript
            transcript  = np.load(os.path.join(self.transcript_dir, transcript_names[i]))[1:-1] # Remove [SOS] and [EOS] from the transcript
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
            # print(transcript.shape)
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        # NOTE:
        # Each mfcc is of shape T1 x 28, T2 x 28, ...
        # Each transcript is of shape (T1+2), (T2+2) before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that
        # the final shape is T x 28 (Where T = T1 + T2 + ...)
        self.mfccs          = np.concatenate(self.mfccs, axis=0)

        # TODO: Concatenate all transcripts in self.transcripts such that
        # the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts    = np.concatenate(self.transcripts, axis=0)
        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        # Take some time to think about what we have done.
        # self.mfcc is an array of the format (Frames x Features).
        # Our goal is to recognize phonemes of each frame
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        # print("self.mfccs", self.mfccs.shape)
        padding = np.zeros((self.context, self.mfccs.shape[1]))
        # print("padding", padding.shape)
        self.mfccs = np.vstack([padding, self.mfccs, padding]) # TODO
        # print("self.mfccs", self.mfccs.shape)

        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such.
        # Hence, we map these phonemes to integers

        # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        phoneme_to_ind = {phoneme: ind for ind, phoneme in enumerate(self.phonemes)}
        self.transcripts = np.array([phoneme_to_ind[phn] for phn in self.transcripts])
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        start = ind
        end = ind + 2*self.context + 1
        frames = self.mfccs[start: end]
        # After slicing, you get an array of shape 2*context+1 x 28. But our MLP needs 1d data and not 2d.
         # Print the shape for debugging
        frames = frames.flatten() # TODO: Flatten to get 1d data

        frames      = torch.FloatTensor(frames) # Convert to tensors
        phonemes    = torch.tensor(self.transcripts[ind])
        return frames, phonemes

In [15]:
class AudioTestDataset(torch.utils.data.Dataset):

    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.

    def __init__(self, root, context=0, partition= "test-clean"): # Feel free to add more arguments

        self.context    = context

        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir       = os.path.join(root, partition, "mfcc")


        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names          = sorted(os.listdir(self.mfcc_dir))

        # # Making sure that we have the same no. of mfcc
        # assert len(mfcc_names) == len(transcript_names)

        self.mfccs = []

        # TODO: Iterate through mfccs
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc        = np.load(os.path.join(self.mfcc_dir, mfcc_names[i]))
            # print(mfcc.shape)
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc = (mfcc - np.mean(mfcc)) / np.std(mfcc)
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)

        # NOTE:
        # Each mfcc is of shape T1 x 28, T2 x 28, ...
        # Each transcript is of shape (T1+2), (T2+2) before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that
        # the final shape is T x 28 (Where T = T1 + T2 + ...)
        self.mfccs          = np.concatenate(self.mfccs, axis=0)

        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        # Take some time to think about what we have done.
        # self.mfcc is an array of the format (Frames x Features).
        # Our goal is to recognize phonemes of each frame
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        # print("self.mfccs", self.mfccs.shape)
        padding = np.zeros((self.context, self.mfccs.shape[1]))
        # print("padding", padding.shape)
        self.mfccs = np.vstack([padding, self.mfccs, padding]) # TODO
        # print("self.mfccs", self.mfccs.shape)


    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        start = ind
        end = ind + 2*self.context + 1
        frames = self.mfccs[start:end]
        # After slicing, you get an array of shape 2*context+1 x 28. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data

        frames      = torch.FloatTensor(frames) # Convert to tensors

        return frames

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments.

In [195]:
config = {
    'epochs'        : 30,
    'batch_size'    : 4096,
    'context'       : 20,
    'init_lr'       : 1e-3,
    'architecture'  : '14',
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
    'use_full_dataset': False,
    'fraction': 0.8,
    'dropout_rate': 0.3
}

# Create Datasets

In [196]:
#TODO: Create a dataset object using the AudioDataset class for the training data
train_data = AudioDataset('11785-f24-hw1p2', context=config['context'])

# TODO: Create a dataset object using the AudioDataset class for the validation data
val_data = AudioDataset('11785-f24-hw1p2', context=config['context'], partition="dev-clean")

# TODO: Create a dataset object using the AudioTestDataset class for the test data
test_data = AudioTestDataset('11785-f24-hw1p2', context=config['context'], partition="test-clean")

In [197]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?
from torch.utils.data import Subset
import numpy as np

# If not using the full dataset, create a subset of the training data
if not config['use_full_dataset']:
  # Create subset dataset
  subset_size = int(config['fraction'] * len(train_data))
  indices = np.random.choice(len(train_data), subset_size, replace=False)
  train_data_subset = Subset(train_data, indices)

  # Use the subset in the DataLoader
  train_loader = torch.utils.data.DataLoader(
    dataset     = train_data_subset,
    num_workers = 4,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True
  )
else:
  # Use the full training dataset
  train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 4,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True
  )


val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*28)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  4096
Context        :  20
Input size     :  1148
Output symbols :  42
Train dataset samples = 36091157, batches = 7050
Validation dataset samples = 1928204, batches = 471
Test dataset samples = 1934138, batches = 473


In [184]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([4096, 1148]) torch.Size([4096])


In [142]:
"""
For HW1 Quiz
"""


from collections import Counter

# Initialize a Counter to store phoneme counts
phoneme_counter = Counter()

# Iterate over the dataset
for frames, phonemes in val_loader:  # phonemes could be a batch of values
    # Update the count for each phoneme in the batch
    phoneme_counter.update(phonemes.tolist())  # Convert the tensor to a list and update counts

# Find the phoneme with the least occurrences
min_phoneme_ind, min_count = phoneme_counter.most_common()[-1]  # last element will have the least count
max_phoneme_ind, max_count = phoneme_counter.most_common()[0]  # first element will have the highest count

ind_to_phoneme = {ind: phoneme for ind, phoneme in enumerate(PHONEMES)}
min_phoneme = ind_to_phoneme[min_phoneme_ind]
max_phoneme = ind_to_phoneme[max_phoneme_ind]
print(f"The phoneme with the least occurrences is {min_phoneme}, with {min_count} occurrences.")
print(f"The phoneme with the least occurrences is {max_phoneme}, with {max_count} occurrences.")

The phoneme with the least occurrences is ZH, with 869 occurrences.
The phoneme with the least occurrences is [SIL], with 319908 occurrences.


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [198]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class Network(torch.nn.Module):

    def __init__(self, input_size, output_size):

        super(Network, self).__init__()
        dropout_rate = config['dropout_rate']
        hidden_size = 2048
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 4096),
            torch.nn.BatchNorm1d(4096),
            torch.nn.GELU(),
            torch.nn.Linear(4096, 2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.GELU(),
            torch.nn.Dropout(dropout_rate),
            torch.nn.Linear(2048, 1024),
            torch.nn.BatchNorm1d(1024),
            torch.nn.GELU(),
            torch.nn.Linear(1024, 1024),
            torch.nn.BatchNorm1d(1024),
            torch.nn.GELU(),
            torch.nn.Dropout(dropout_rate),
            torch.nn.Linear(1024, 512),
            torch.nn.BatchNorm1d(512),
            torch.nn.GELU(),
            torch.nn.Linear(512, 512),
            torch.nn.BatchNorm1d(512),
            torch.nn.GELU(),
            torch.nn.Dropout(dropout_rate),
            torch.nn.Linear(512, 512),
            torch.nn.BatchNorm1d(512),
            torch.nn.GELU(),
            torch.nn.Linear(512, output_size)
        )

    def forward(self, x):
        out = self.model(x)

        return out

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler.

In [199]:
INPUT_SIZE  = (2*config['context'] + 1) * 28 # Why is this the case? (flattened?)
model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
summary(model, frames.to(device))
# Check number of parameters of your network
# Remember, you are limited to 20 million parameters for HW1 (including ensembles)

----------------------------------------------------------------------------------------------------
Layer                   Kernel Shape         Output Shape         # Params (K)      # Mult-Adds (M)
0_Linear                [1148, 4096]         [4096, 4096]             4,706.30                 4.70
1_BatchNorm1d                 [4096]         [4096, 4096]                 8.19                 0.00
2_GELU                             -         [4096, 4096]                    -                    -
3_Linear                [4096, 2048]         [4096, 2048]             8,390.66                 8.39
4_BatchNorm1d                 [2048]         [4096, 2048]                 4.10                 0.00
5_GELU                             -         [4096, 2048]                    -                    -
6_Dropout                          -         [4096, 2048]                    -                    -
7_Linear                [2048, 1024]         [4096, 1024]             2,098.18                 2.10

In [200]:
criterion = torch.nn.CrossEntropyLoss() # Defining Loss function.
# We use CE because the task is multi-class classification

optimizer = torch.optim.AdamW(model.parameters(), lr= config['init_lr']) #Defining Optimizer
# Recommended : Define Scheduler for Learning Rate,
# including but not limited to StepLR, MultiStep, CosineAnnealing, CosineAnnealingWithWarmRestarts, ReduceLROnPlateau, etc.
# You can refer to Pytorch documentation for more information on how to use them.
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',     # Use 'min' for validation loss, 'max' for accuracy
    factor=0.01,     # Reduce learning rate by a factor of 0.1
    patience=10)

# Is your training time very high?
# Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [201]:
torch.cuda.empty_cache()
gc.collect()

2839

In [202]:
def train(model, dataloader, optimizer, criterion, scheduler=None):

    model.train()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Move Data to Device (Ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        ### Forward Propagation
        logits  = model(frames)

        ### Loss Calculation
        loss    = criterion(logits, phonemes)

        ### Backward Propagation
        loss.backward()

        ### Gradient Descent
        optimizer.step()

        tloss   += loss.item()
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    tloss   /= len(train_loader)
    tacc    /= len(train_loader)

    if scheduler:
        scheduler.step(tloss)

    return tloss, tacc

In [203]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode():
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))),
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(val_loader)
    vacc    /= len(val_loader)

    return vloss, vacc

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb.

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [204]:
wandb.login(key="4c68592a14802648ff3220711294d6e5fe7d272f") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [205]:
# Create your wandb run
run = wandb.init(
    name    = f"arch-{config['architecture']}-run-2", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    #id     = "y28t31uz", ### Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2", ### Project should be created in your wandb account
    config  = config ### Wandb Config for your run
)

In [206]:
### Save your model architecture as a string with str(model)
model_arch  = str(model)

### Save it in a txt file
arch_file   = open("model_arch.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/content/wandb/run-20240920_191342-qxc2rzcg/files/model_arch.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [207]:
# Iterate over number of epochs to train and evaluate your model
best_val_acc = 0.0
torch.cuda.empty_cache()
gc.collect()
wandb.watch(model, log="all")

# Extract architecture number from config
architecture_number = config['architecture']

for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion, scheduler)
    val_loss, val_acc       = eval(model, val_loader)

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    ### Log metrics at each epoch in your run
    # Optionally, you can log at each batch inside train/eval functions
    # (explore wandb documentation/wandb recitation)
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss,
               'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})

    ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best
    if val_acc > best_val_acc:
        print(f"Validation accuracy improved from {best_val_acc*100:.04f}% to {val_acc*100:.04f}%! Saving model...")
        best_val_acc = val_acc
        # Save the best model checkpoint with all necessary information
        torch.save({
            'epoch': epoch + 1,  # Save the current epoch number
            'architecture': architecture_number,
            'model_state_dict': model.state_dict(),  # Save the model state dict
            'optimizer_state_dict': optimizer.state_dict(),  # Save the optimizer state dict
            'val_acc': val_acc  # Save the best validation accuracy
        }, 'best_model.pth')

        # Save the best model to WandB
        wandb.save('best_model.pth')

    # Save the latest model checkpoint after every epoch
    torch.save({
        'epoch': epoch + 1,
        'architecture': architecture_number,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'val_acc': val_acc
    }, 'latest_model.pth')

    # Save the latest model to WandB
    wandb.save('latest_model.pth')



Epoch 1/30


Train:   0%|          | 0/7050 [00:00<?, ?it/s]

Val:   0%|          | 0/471 [00:00<?, ?it/s]

	Train Acc 76.4301%	Train Loss 0.7272	 Learning Rate 0.0010000
	Val Acc 78.6653%	Val Loss 0.6435
Validation accuracy improved from 0.0000% to 78.6653%! Saving model...

Epoch 2/30


Train:   0%|          | 0/7050 [00:00<?, ?it/s]

...

Val:   0%|          | 0/471 [00:00<?, ?it/s]

	Train Acc 90.8524%	Train Loss 0.2476	 Learning Rate 0.0010000
	Val Acc 80.7860%	Val Loss 0.7168

Epoch 30/30


Train:   0%|          | 0/7050 [00:00<?, ?it/s]

Val:   0%|          | 0/471 [00:00<?, ?it/s]

	Train Acc 90.8828%	Train Loss 0.2465	 Learning Rate 0.0010000
	Val Acc 80.7692%	Val Loss 0.7175


In [ ]:
# wandb.init(project='hw1p2', id='arch-13-run-2', resume='allow')

# Download the model checkpoint file from WandB
checkpoint_file = 'best_model-2.pth'  # Replace with the path to your uploaded file
print(f"Checkpoint downloaded: {checkpoint_file}")

In [ ]:
# Inspect the checkpoint file
checkpoint = torch.load(checkpoint_file)
print("Checkpoint keys:", checkpoint.keys())
# Load the model directly with state_dict
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
checkpoint = torch.load(checkpoint_file, weights_only=True)
model.load_state_dict(checkpoint)
start_epoch = 13
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

print(f"Model and optimizer loaded successfully from epoch {start_epoch}.")

# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [208]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval() # TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    # Reverse dictionary to map indices back to phonemes
    ind_to_phoneme = {ind: phoneme for ind, phoneme in enumerate(PHONEMES)}

    ### Which mode do you need to avoid gradients?
    with torch.inference_mode(): # TODO

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)

            logits  = model(mfccs)

            ### Get most likely predicted phoneme index with argmax
            predicted_indices = torch.argmax(logits, dim=1)

            # Convert predicted indices to corresponding phonemes
            predicted_phonemes = [ind_to_phoneme[idx.item()] for idx in predicted_indices]

            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval
            test_predictions.extend(predicted_phonemes)

    return test_predictions

In [209]:
total_samples = len(test_loader.dataset)
print("Total number of samples:", total_samples)

Total number of samples: 1934138


In [210]:
predictions = test(model, test_loader)

  0%|          | 0/473 [00:00<?, ?it/s]

In [211]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [212]:
### Finish your wandb run
run.finish()

lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▄▅▆▆▇▇▇▇▇▇▇▇█████████████████
train_loss,█▅▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▆▇███████████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
valid_loss,▄▁▁▁▂▂▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇█▇▇▇███
lr,0.001
train_acc,90.88281
train_loss,0.24646
val_acc,80.76919
valid_loss,0.71746


In [213]:
### Submit to kaggle competition using kaggle API (Uncomment below to use)
!kaggle competitions submit -c 11785-hw1p2-f24 -f ./submission.csv -m "Test Submission"

### However, its always safer to download the csv file and then upload to kaggle

100% 19.4M/19.4M [00:03<00:00, 6.14MB/s]
Successfully submitted to 11785 HW1P2 Fall 2024